# Задание 1

Напишите программу с двумя потоками. В первом потоке выведите все чётные числа, во втором -- нечётные. Нижняя и верхняя граница указывается на входе (можно захардкодить, чтобы не вводить).

Пример:

Входные данные:
```
30
50
```

Ответ:
```
List of even numbers:
30 32 34 36 38 40 42 44 46 48 50
List of odd numbers:
31 33 35 37 39 41 43 45 47 49
```

In [2]:
import threading

def print_even_numbers(start, end):
    print("List of even numbers:")
    for number in range(start, end + 1):
        if number % 2 == 0:
            print(number, end=" ")
    print()

def print_odd_numbers(start, end):
    print("Список нечётных чисел:")
    for number in range(start, end + 1):
        if number % 2 != 0:
            print(number, end=" ")
    print()

if __name__ == "__main__":
    # Захардкоженные границы
    start = 30
    end = 50

    # Создаем два потока
    even_thread = threading.Thread(target=print_even_numbers, args=(start, end))
    odd_thread = threading.Thread(target=print_odd_numbers, args=(start, end))

    # Запускаем потоки
    even_thread.start()
    odd_thread.start()

    # Ожидаем завершения потоков
    even_thread.join()
    odd_thread.join()

    print("Оба потока завершили выполнение.")


List of even numbers:Список нечётных чисел:

3130 33 32 34 36 38 40 42 44 46 48 50 
 35 37 39 41 43 45 47 49 
Оба потока завершили выполнение.


# Задание 2

Реализуйте программу, которая конкурентно посчитает факториал числа.

Входные данные:
```
12
```

Ответ:
```
479001600
```

In [3]:
import threading

def calculate_factorial(number):
    result = 1
    for i in range(1, number + 1):
        result *= i
    print(f"Factorial of {number} is: {result}")

if __name__ == "__main__":
    number = 12

    # Создаем поток для вычисления факториала
    factorial_thread = threading.Thread(target=calculate_factorial, args=(number,))

    # Запускаем поток
    factorial_thread.start()

    # Ожидаем завершения потока
    factorial_thread.join()


Factorial of 12 is: 479001600


# Задание 3

Напишите 2 функции: первая считает квадрат числа (можно взять готовые реализации), вторая -- сумму чисел.

Каждую из указанных функций запустите в отдельном процессе. В результате у вас должен получится пайплайн, который получает на вход числа от 1 до $n$ (включительно), считает для них квадрат, и для полученных чисел считает сумму.

Входные данные:
```
10
```

Ответ:
```
385
```

In [4]:
import multiprocessing

def calculate_square(numbers, conn):
    squares = [number ** 2 for number in numbers]
    conn.send(squares)
    conn.close()

def calculate_sum(conn):
    squares = conn.recv()
    total_sum = sum(squares)
    print(f"The sum of squares is: {total_sum}")

if __name__ == "__main__":
    n = 10  # Верхняя граница
    numbers = list(range(1, n + 1))

    # Создаем канал связи
    parent_conn, child_conn = multiprocessing.Pipe()

    # Создаем процессы
    square_process = multiprocessing.Process(target=calculate_square, args=(numbers, child_conn))
    sum_process = multiprocessing.Process(target=calculate_sum, args=(parent_conn,))

    # Запускаем процессы
    square_process.start()
    sum_process.start()

    # Ожидаем завершения процессов
    square_process.join()
    sum_process.join()


The sum of squares is: 385


# Задание 4

Реализуйте параллельный алгоритм сортировки слиянием (merge sort) с использованием модуля multiprocessing.

Входные данные:
```
3 14 15 9 2 6 5 35
```

Ответ:
```
2 3 5 6 9 14 15 35
```

In [5]:
import multiprocessing

# Функция для слияния двух отсортированных массивов
def merge(left, right):
    result = []
    i = j = 0

    # Пока оба массива не пусты, добавляем меньший элемент
    while i < len(left) and j < len(right):
        if left[i] <= right[j]:
            result.append(left[i])
            i += 1
        else:
            result.append(right[j])
            j += 1

    # Добавляем оставшиеся элементы из левого и правого массива
    result.extend(left[i:])
    result.extend(right[j:])
    return result

# Функция для последовательной сортировки слиянием
def merge_sort(data):
    if len(data) <= 1:
        return data

    mid = len(data) // 2
    left = merge_sort(data[:mid])
    right = merge_sort(data[mid:])
    return merge(left, right)

# Функция для параллельной сортировки слиянием с использованием multiprocessing
def parallel_merge_sort(data):
    if len(data) <= 1:
        return data

    mid = len(data) // 2

    # Создаем пул процессов для параллельной сортировки левой и правой части
    with multiprocessing.Pool(processes=2) as pool:
        left, right = pool.map(merge_sort, [data[:mid], data[mid:]])

    # Объединяем отсортированные части
    return merge(left, right)

if __name__ == "__main__":
    # Тестовые данные
    input_data = [3, 14, 15, 9, 2, 6, 5, 35]

    # Выполняем параллельную сортировку
    sorted_data = parallel_merge_sort(input_data)

    # Вывод результата
    print("Sorted array:", sorted_data)


Sorted array: [2, 3, 5, 6, 9, 14, 15, 35]
